# Creating a Chess AI using Tensorflow

In [111]:
import chess
import chess.engine
import numpy as np
import random
import tensorflow as tf

In [168]:
def create_random_board(max_depth = 200):
    depth = random.randrange(0, max_depth)
    board = chess.Board()
    for _ in range(depth):
        move = random.choice(list(board.legal_moves))
        board.push(move)
        if board.is_game_over():
            break
    return board

def sf_evaluate(board, depth = 0):
    sf = chess.engine.SimpleEngine.popen_uci("stockfish")
    analysis = sf.analyse(board, chess.engine.Limit(depth=depth))
    return analysis['score'].white().score()

def create_dataset(n = 10000):
    positions = np.empty(0, dtype = object)
    evals = np.empty(0, dtype = numpy.int16)
    for _ in range(n):
        board = create_random_board()
        evaluation = sf_evaluate(board)
        if evaluation:
            positions = np.append(positions, board)
            evals = np.append(evals, evaluation)
    np.save('positions', positions)
    np.save('evals', evals)

In [169]:
%%time
create_dataset(10)

CPU times: user 484 ms, sys: 221 ms, total: 705 ms
Wall time: 5.35 s
